<a href="https://colab.research.google.com/github/ceresisonline/youtube-to-birdsong/blob/main/youtube_to_birdsong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Dependencies


In [ ]:
!git clone https://github.com/ceresisonline/youtube-to-birdsong

!apt install sox

!sudo -H pip install --upgrade youtube-dl

!chmod +x /content/youtube-to-birdsong/csdr-install.sh

!pip install ffmpeg-normalize

!/content/youtube-to-birdsong/csdr-install.sh

# Content Generation


In [ ]:
#delete files from previous executions 
!rm -rf *.wav *.mp3 *.mp4 *.prof *.mov

#@markdown Insert the link to the YouTube video you want to download & make into birdsong. Try to find a slow paced video and avoid avoid music videos.

# Björk talking about her TV: https://www.youtube.com/watch?v=75WFTHpOw8Y
YouTube_Link= "https://www.youtube.com/watch?v=75WFTHpOw8Y" #@param {type:"string"}   

#@markdown Set resolution of downloaded video. YouTube downloads take a while, so keep the max resolution low if possible.
MaxVideoHeight = "720" #@param {type:"string"}  

#@markdown Reduce background noise in the video. Don't change this if you don't know what you're doing. Try not to go below 0.2 or above 0.3
Static_Reduce = "0.28" #@param {type:"string"}    

#download video
!youtube-dl -f "bestvideo[height<={MaxVideoHeight}]+bestaudio/best[height<={MaxVideoHeight}]" --recode-video mp4 -o vid {YouTube_Link}

#save audio from video
!ffmpeg -i vid.mp4 -vn cc-raw.wav

#Generate a sample of noise from the audio of the file:
#-t duration: the duration (h:m:s.ms) of audio segment to cut that represents the noise.
!ffmpeg -i cc-raw.wav -acodec pcm_s16le -ar 128k -vn -ss 00:00:00.0 -t 00:00:01.0 noiseaud.wav

#Generate a noise profile in sox:
!sox noiseaud.wav -n noiseprof noise.prof

#Clean the noise samples from the audio stream:
!sox cc-raw.wav cc.wav noisered noise.prof {Static_Reduce}

#convert audio from video into birdsong
#requires soX and csdr
!perl /content/youtube-to-birdsong/blackbird.pl -n

!ffmpeg-normalize birdsong1.wav -o birdsong2.wav

#add birdsounds to original video
!ffmpeg -i vid.mp4 -i birdsong2.wav -map 0:v -map 1:a -c:v copy -shortest vid+newaudio.mp4

# View new generated video with blackbird audio

In [ ]:
from IPython.display import HTML
from base64 import b64encode

print ("Download this at /content/vid+newaudio.mp4" )

mp4 = open('/content/vid+newaudio.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)



In [ ]:
import librosa
import IPython.display as ipd

print ("Download this at /content/birdsong2.wav" )

print ("New Audio:")

x, sr = librosa.load('birdsong2.wav') 
ipd.Audio(x, rate=sr)

# View downloaded video with original audio




In [ ]:
from IPython.display import HTML
from base64 import b64encode

print ("Download this at /content/vid.mp4" )

mp4 = open('/content/vid.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
import librosa
import IPython.display as ipd

print ("Download this at /content/cc-raw.wav" )

print ("Original video audio:")

x, sr = librosa.load('cc-raw.wav') 
ipd.Audio(x, rate=sr)

# Download generated content

In [ ]:
#@markdown Check these boxes to download the generated content. Make sure you have allowed Google Colab to download multiple files in your browser settings.

#@markdown Downloads from Google Colab tend to take a while, so I would suggest previewing it with the cells above to see if you really want it.
import os
from google.colab import files

SAVE_BLACKBIRD_VIDEO = True #@param {type:"boolean"}
SAVE_BLACKBIRD_AUDIO = True #@param {type:"boolean"}
SAVE_ORIGINAL_VIDEO = False #@param {type:"boolean"}
SAVE_ORIGINAL_AUDIO = False #@param {type:"boolean"}

if SAVE_BLACKBIRD_VIDEO:
  os.rename('vid+newaudio.mp4', 'blackbird_video.mp4')
  from google.colab import files
  files.download( "/content/blackbird_video.mp4" ) 

if SAVE_BLACKBIRD_AUDIO:
  from google.colab import files
  os.rename('birdsong2.wav', 'blackbird_audio.mp4')
  files.download( "/content/blackbird_audio.mp4" ) 

if SAVE_ORIGINAL_VIDEO:
  from google.colab import files
  os.rename('vid.mp4', 'original_video.mp4')
  files.download( "/content/original_video.mp4" ) 

if SAVE_ORIGINAL_AUDIO:
  from google.colab import files
  os.rename('cc-raw', 'original_audio.wav')
  files.download( "/content/original_audio.wav" ) 